# Na aula anterior...

Qual a importância de realizar um grid search?

Qual utilidade das Pipelines do sklearn dentro do universo de Ciência de Dados?

In [3]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier

# Hoje!

**Para vocês**: otimizem os hiperparâmetros de um dos modelos de vocês.

In [4]:

df = pd.read_csv("C:/Users/Avell/Google Drive/Luiz Gabriel - PC/06.Material de Cursos/LetsCode_DS_Curso/07_Machine_Learning_II/dados/titanic.csv")

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
var_expl = [
    'Pclass', 
    'Sex', 
    'Age',
    'SibSp',
    'Parch', 
    'Fare',
    'Embarked'
]

cat_vars = [
    'Sex',
    'Embarked'
]

num_vars = [
    'Pclass', 
    'Age',
    'SibSp',
    'Parch', 
    'Fare'
]

var_resp = 'Survived'

In [7]:
df.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
df_treino, df_teste = train_test_split(df.drop(columns="PassengerId"), test_size = 0.15, random_state = 1)

In [9]:
x_treino = df_treino[var_expl].copy()
y_treino = df_treino[var_resp].copy()

x_teste = df_teste[var_expl].copy()
y_teste = df_teste[var_resp].copy()


In [10]:
%%time

parametros = {
    'modelo__num_leaves': list(np.arange(3,20,3)),
    'modelo__max_depth': list(np.arange(3,20,3)),
    'modelo__n_estimators': list(np.arange(100,250,50)),
    'modelo__learning_rate': [0.01, 0.1],
}

modelo = LGBMClassifier(
    colsample_bytree = 0.5, 
    subsample = 0.5,
    subsample_freq = 5,
    random_state = 1
)


pipe_encoder = Pipeline(steps = [
    ('pipe_encoder', OneHotEncoder())
])

pipe_selecao = ColumnTransformer(transformers = [
    ('pipe_encoder', pipe_encoder, cat_vars),
    ('outras_colunas', 'passthrough', num_vars),
])

pipeline = Pipeline(steps = [
    ('preproc', pipe_selecao),
    ('modelo', modelo)
])

gscv = GridSearchCV(
    estimator = pipeline,
    param_grid = parametros,
    scoring = 'roc_auc',
    refit = True,
    cv = 3,
    verbose = 3
)

gscv.fit(x_treino, y_treino)
y_pred_treino = gscv.predict_proba(x_treino)[:, 1]
y_pred_teste = gscv.predict_proba(x_teste)[:, 1]

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV 1/3] END modelo__learning_rate=0.01, modelo__max_depth=3, modelo__n_estimators=100, modelo__num_leaves=3;, score=0.822 total time=   0.0s
[CV 2/3] END modelo__learning_rate=0.01, modelo__max_depth=3, modelo__n_estimators=100, modelo__num_leaves=3;, score=0.867 total time=   0.0s
[CV 3/3] END modelo__learning_rate=0.01, modelo__max_depth=3, modelo__n_estimators=100, modelo__num_leaves=3;, score=0.855 total time=   0.0s
[CV 1/3] END modelo__learning_rate=0.01, modelo__max_depth=3, modelo__n_estimators=100, modelo__num_leaves=6;, score=0.831 total time=   0.0s
[CV 2/3] END modelo__learning_rate=0.01, modelo__max_depth=3, modelo__n_estimators=100, modelo__num_leaves=6;, score=0.879 total time=   0.0s
[CV 3/3] END modelo__learning_rate=0.01, modelo__max_depth=3, modelo__n_estimators=100, modelo__num_leaves=6;, score=0.863 total time=   0.0s
[CV 1/3] END modelo__learning_rate=0.01, modelo__max_depth=3, modelo__n_estimators=10

In [44]:
roc_auc_score(y_treino, y_pred_treino)

0.945536454631604

In [45]:
roc_auc_score(y_teste, y_pred_teste)

0.8650462962962963

In [ ]:
print("Treino: ", roc_auc_score(y_treino, y_pred_treino))
print("Teste: ", roc_auc_score(y_teste, y_pred_teste))

In [52]:
!pip install tune-sklearn ray[tune]

  Using cached aiosignal-1.2.0-py3-none-any.whl (8.2 kB)


You should consider upgrading via the 'c:\users\avell\anaconda3\python.exe -m pip install --upgrade pip' command.


In [56]:
from tune_sklearn import TuneSearchCV

In [1]:
gs = TuneSearchCV(pipeline, parametros,  n_trials=5,   scoring="roc_auc")
gs.fit(x_treino, y_treino)
print(gs.cv_results_)


NameError: name 'TuneSearchCV' is not defined

In [43]:
gscv.best_params_

{'modelo__learning_rate': 0.05,
 'modelo__max_depth': 5,
 'modelo__n_estimators': 250,
 'modelo__num_leaves': 15}

In [ ]:
gscv.get_params

**Para vocês**: organizem o projeto de vocês para que o pré-processamento e o modelo esteja no formato de pipeline.

## SVM

- Material complementar: [Aula do professor Patrick Winston](https://www.youtube.com/watch?v=_PwhiWxHK8o)
- Todos os modelos de classificação possuem um [fronteira de decisão](https://ml-playground.com/).
- Algoritmos já vistos não são construídas em torno desse conceito. 
- O objetivo do SVM é encontrar uma fronteira que consiga separar o conjunto de dados em classes positivas e negativas
- Em um plano, existe um conjunto infinito de linhas que podem seperar duas classes
- Mas, qual o melhor?

<center><img src="https://s3-sa-east-1.amazonaws.com/lcpi/fceb4ea0-9de8-4048-9cc4-c9d4355b671c.png" width = 25%/></center>

- Aplicando com sklearn.

- Mostrando um caso linear.

- Esse algoritmo, apesar de lindo, é pouco prático as is, porque traça fronteiras lineares, mas o mundo não é linear. Então, durante 20 anos, o SVM ficou "engavatado" até que a ideia do Kernel Trick apareceu.